<a href="https://colab.research.google.com/github/JacobSchmitt/Basaa-Machine-Learning/blob/main/basaa/test/Basaa_classic_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload the data and run the baseline

In [ ]:
%%capture
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/train.py
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/convert.py
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/Makefile
!wget -P '/content/data' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/train.tsv 
!wget -P '/content/data' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/dev.tsv 

In [ ]:
!wget -q https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/train.tsv 
!wget -q https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/dev.tsv 
!wget -q https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/evaluate.py
!wget -q https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/train.py
!wget -q https://raw.githubusercontent.com/ftyers/global-classroom/main/basaa/data/test/test.tsv

In [ ]:
%%bash
cd baseline
make

python3 train.py ../data/train.tsv model.tsv
Stored 14596 words
python3 convert.py model.tsv < ../data/dev.tsv > output.tsv
python3 ../evaluate.py ../data/dev.tsv output.tsv
CER: 16.54687852515551
WER: 40.88336922205728


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
train = pd.read_csv('train.tsv', sep ='\t', names = ['Protestant', 'Academy'])
valid = pd.read_csv('dev.tsv', sep ='\t', names = ['Protestant', 'Academy'])
test = pd.read_csv('test.tsv', sep ='\t', names = ['Protestant', 'Academy'])

# Create the dictionaries of most common pairs

In [ ]:
!python3 train.py train.tsv t.tsv

Stored 14596 words


In [ ]:
t = pd.read_csv('t.tsv', sep ='\t', names = ['var', 'prot', 'ac'])

In [ ]:
d = t[t['var'] > 0.8].drop('var', axis=1)
d_p = d.prot.to_list()
d_a = d.ac.to_list()
no_change = {d_p[i]:d_a[i] for i in range(len(d_p))}

In [ ]:
no_change['...Ni'] = '...Ni' 
no_change['Ni'] = 'Ni' 
no_change['U'] = 'U' 
no_change['ntibil'] = 'ntı̄bil'
no_change['mbôô'] = 'mɓoô'
no_change['Tôôs'] = 'Toôs'
no_change['tôôs'] = 'toôs'
no_change['Kôôt'] = 'Koôt'
no_change['kôôt'] = 'koôt'
no_change['mapep.'] = 'màpɛp.'
no_change['mbéndha'] = 'm̂ɓêndhā'

# Write the rules that will be applied  before the predictions (there is another set for 'post-prediction')

In [ ]:
def rules(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  
  for _ in range(10):
    if x == 'ba':
      x = x.replace('ba', 'ɓa')
    elif x == 'bo':
      x = x.replace('bo', 'ɓɔ')
    elif x == 'yom':
      x = x.replace('yom', 'yɔm')
    elif 'tjo' in x:
      x = x.replace('tjo', 'cɔ')
    elif 'tj' in x:
      x = x.replace('tj', 'c')   
    elif 'ñ' in x:
      x = x.replace('ñ', 'ŋ')
    elif double_o in x:
      x = x.replace(double_o, 'o')
    elif double_e in x:
      x = x.replace(double_e, 'e')
    elif 'e' in x:
      x = x.replace('e', 'ɛ')
    elif x.startswith(('b', 'm')):
      x = x.replace('b', 'ɓ')
    
  return x

# BPE

In [ ]:
selected_punc =  '«»—…“”\'":;,....\|/()+!?[]=-1234567890'

In [ ]:
def smart_split(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  x = x.replace('é', double_e).replace('ô', double_o).replace('ö', double_o_dots)
  word = x
  l = list(x)
  
  vowels = ['ı', 'ŋ', 'ɔ', 'ɛ', 'ñ', 'n', 'a', 'i', 'o', 'e', 'u', 'y']
  diacritics = ('̀', '́', '̂', '̄', '̌')
  res = []
  s = ''
  i = 0
  while i < len(l):
    if l[i] not in vowels and l[i] not in diacritics:
      s += l[i]
    elif l[i] in vowels:
      if x[i:i+2].endswith(diacritics):
        s += l[i]+l[i+1]
        if i+2 == len(l)-1 and l[i+2] not in vowels:
          s+= l[i+2]
        res.append(s)
        i += 1
        s = ''          
      else:
        s += l[i]
        if i+1 == len(l)-1 and l[i+1] not in vowels:
          s+= l[i+1]
        res.append(s)
        s = ''
    i += 1
  if res:
    res[0] = '_'+res[0]
    res[-1] = res[-1]+'_'
  return word, res

In [ ]:
train = train.dropna()

In [ ]:
train = train.drop(train[train.Protestant == '?'].index)
valid = valid.drop(valid[valid.Protestant == '?'].index)
test = test.drop(test[test.Protestant == '??'].index)

In [ ]:
word1 =[]
word2 =[]

res = []
res1 = []

for i in range(len(train)):
  line1 = train.Protestant.iloc[i].split()
  line2 = train.Academy.iloc[i].split()
  
  for i in range(len(line1)):
    if line1[i] in no_change and line2[i] == no_change[line1[i]]:
      continue
    else:
      cl_w = rules(str(line1[i]).strip(selected_punc).lower())
      cl_t = str(line2[i]).strip(selected_punc).lower()
      w1, x = smart_split(cl_w)
      w2, y = smart_split(cl_t)
      if len(x) == len(y):
        for _ in range(len(x)):
          word1.append(w1)
          word2.append(w2)
        res.extend(x)
        res1.extend(y)
      else:
        print('error in line', i, '\n', x, '\n', y)

In [ ]:
sylls = pd.DataFrame({'input': res, 'output': res1})

In [ ]:
sylls.to_csv('syll_pairs.tsv', index=False, sep='\t', header=False)

In [ ]:
!python3 train.py syll_pairs.tsv syll.tsv

Stored 903 words


In [ ]:
t_syll = pd.read_csv('syll.tsv', sep ='\t', names = ['var', 'prot', 'ac'])

In [ ]:
d_syll = t_syll[t_syll['var'] > 0.8].drop('var', axis=1)
d_syll_p = d_syll.prot.to_list()
d_syll_a = d_syll.ac.to_list()
no_change_syll = {d_syll_p[i]:d_syll_a[i] for i in range(len(d_syll_p))}

# Let's save most common syllables to the dictionary and remove them from input_target

In [ ]:
input_target = pd.DataFrame({'input': res, 'output': res1, 'word_inp': word1, 'word_tgt': word2})

In [ ]:
def sel(x):
  if x in no_change_syll:
    return False
  else:
    return True

In [ ]:
input_target['filt'] = input_target.input.apply(sel)

input_target = input_target[input_target.filt == True]
input_target = input_target.drop('filt', axis=1)

In [ ]:
input_target

,input,output,word_inp,word_tgt
0,_mɛ_,_mɛ̀_,mɛ,mɛ̀
1,_ɓot_,_ɓɔ̀t_,ɓot,ɓɔ̀t
2,_ɓô,_ɓo,ɓôdaa,ɓodàa
4,a_,a_,ɓôdaa,ɓodàa
5,_n,_n,ni,ni
...,...,...,...,...
54523,_mɓus_,_mbūs_,mɓus,mbūs
54524,_ma,_ma,man,man
54526,_ka_,_kà_,ka,kà
54527,_i_,_ı̀_,i,ı̀


In [ ]:
#repeated lines
input_target.groupby(input_target.columns.tolist()).size().reset_index().\
    rename(columns={0:'records'}).sort_values(by=['records'], ascending=False)

,input,output,word_inp,word_tgt,records
388,_i_,_i_,i,i,2049
390,_i_,_ı̀_,i,ı̀,789
2048,_mɛ_,_mɛ̀_,mɛ,mɛ̀,692
3509,_u_,_ù_,u,ù,681
6972,i_,ı̀_,ni,nı̀,584
...,...,...,...,...,...
4192,_ɓa,_ɓâ,ɓaam,ɓâàm,1
4191,_ɓa,_ɓà,ɓaɛmɓ,ɓàɛ̀mb,1
4190,_ɓa,_ɓà,ɓaɓôŋôl,ɓàɓòŋòl,1
4189,_ɓa,_ɓà,ɓaɓoɓla,ɓàɓɔ̀blà,1


In [ ]:
# input equals output
input_target[input_target.input == input_target.output].groupby(input_target.columns.tolist()).size().reset_index().\
    rename(columns={0:'records'}).sort_values(by=['records'], ascending=False)

,input,output,word_inp,word_tgt,records
136,_i_,_i_,i,i,2049
774,_n,_n,ni,nı̀,584
599,_mɛ_,_mɛ_,mɛ,mɛ,580
1382,_u_,_u_,u,u,560
772,_n,_n,ni,ni,211
...,...,...,...,...,...
1412,_ŋ,_ŋ,ŋéŋlaha,ŋeŋlaha,1
1413,_ŋ,_ŋ,ŋéŋlana,ŋèŋlana,1
1415,_ŋ,_ŋ,ŋgaa,ŋgaà,1
1418,_ŋ,_ŋ,ŋgaba,ŋgabà,1


# Encode tones as labels

In [ ]:
input_target.loc[input_target.output.str.contains('̀'), 'label'] = 1
input_target.loc[input_target.output.str.contains('́'), 'label'] = 2
input_target.loc[input_target.output.str.contains('̂'), 'label'] = 3
input_target.loc[input_target.output.str.contains('̄'), 'label'] = 4
input_target.loc[input_target.output.str.contains('̌'), 'label'] = 5

input_target.label = input_target.label.fillna(0)
input_target.label = input_target.label.astype(int)

# Make vectors

In [ ]:
def prev_diacritic(word, syll):
  prev_d = [0, 0, 0, 0, 0]
  diacritics = ['̀', '́', '̂', '̄', '̌']
  s = syll.replace('_', '')
  w = word.split(s)
  if w:
    for i in range(len(diacritics)):
      if diacritics[i] in w[0]:
        prev_d[i] = 1
  return prev_d

In [ ]:
# def next_diacritic(word, syll):
#   next_d = [0, 0, 0, 0, 0]
#   diacritics = ['̀', '́', '̂', '̄', '̌']
#   s = syll.replace('_', '')
#   w = word.split(s)
#   w.remove(w[0])
#   w = ''.join(w)
#   if len(w) > 1:
#     for i in range(len(diacritics)):
#       if diacritics[i] in w:
#         next_d[i] = 1
#   return next_d

In [ ]:
input_target.head()

,input,output,word_inp,word_tgt,label
0,_mɛ_,_mɛ̀_,mɛ,mɛ̀,1
1,_ɓot_,_ɓɔ̀t_,ɓot,ɓɔ̀t,1
2,_ɓô,_ɓo,ɓôdaa,ɓodàa,0
4,a_,a_,ɓôdaa,ɓodàa,0
5,_n,_n,ni,ni,0


In [ ]:
input_target['prev'] = input_target.apply(lambda x: prev_diacritic(x['word_tgt'], x['output']), axis=1)

In [ ]:
# input_target['next'] = input_target.apply(lambda x: next_diacritic(x['word_inp'], x['input']), axis=1)

In [ ]:
def features(x):
  #feature1 - start 
  #feature2 - end
  #feature3 - middle
  vect = [0, 0, 0]
  if len(x) == 1:
    vect[2] = 1
    return vect
  elif len(x) > 1:
    if x.startswith('_'):
      vect[0] = 1
      if x.endswith('_'):
        vect[1] = 1
        return vect
      else:
        vect[2] = 1
        return vect
    else:
      if x.endswith('_'):
        vect[1] = 1
        vect[2] = 1
        return vect
      else:
        vect[2] = 1
        return vect
  return vect     

In [ ]:
def vowel_features(x):
  #feature1 - starts with vowel 
  #feature2 - ends with vowel
  vowels = ['ı', 'ŋ', 'ɔ', 'ɛ', 'ñ', 'n', 'a', 'i', 'o', 'e', 'u', 'y']
  vect = [0, 0]

  if len(x) == 1:
    if x in vowels:
      vect[1] = 1
      return vect
  elif len(x) == 2 and x[0] in vowels:
    vect[0] = 1
    vect[1] = 1
    return vect
  elif x.startswith('_'):
    if x[1] in vowels:
      vect[0] = 1
      if x.endswith('_'):
        if x[-2] in vowels:
          vect[1] = 1
          return vect
      else:
        if x[-1] in vowels:
          vect[1] = 1
          return vect
    elif x.endswith('_'):
      if x[-2] in vowels:
        vect[1] = 1
        return vect
    else:
      if x[-1] in vowels:
        vect[1] = 1
        return vect
  else:
    if x[0] in vowels:
      vect[0] = 1
      if x.endswith('_'):
        if x[-2] in vowels:
          vect[1] = 1
          return vect
      else:
        if x[-1] in vowels:
          vect[1] = 1
          return vect
    elif x.endswith('_'):
        if x[-2] in vowels:
          vect[1] = 1
          return vect
    else:
      if x[-1] in vowels:
        vect[1] = 1
        return vect
  return vect

In [ ]:
# def r_or_w(x):
#   # feature1 = r
#   # feature2 = w
#   vect = [0, 0]
#   if 'r' in x:
#     vect[0] = 1
#   if 'w' in x:
#     vect[1] = 1
#   return vect

In [ ]:
# def j(x):
#   if 'j' in x:
#     return 1
#   else:
#     return 0

In [ ]:
def plosives(x):
  plosives = ['t','d','k','ɡ','p','b']
  vect = [0, 0, 0, 0, 0, 0]
  for i in range(len(plosives)):
    if plosives[i] in x:
      vect[i] = 1
  return vect

In [ ]:
# def m(x):  
#   if 'm' in x:
#     return 1
#   else:
#     return 0

In [ ]:
def close(x): 
  vect = 0
  vow = ['ı', 'ŋ', 'ɔ', 'ɛ', 'ñ', 'n', 'a', 'i', 'o', 'e', 'u', 'y']
  
  for i in range(len(vow)):
    if x.endswith('_'):
      if x[-2] == vow[i]:
        vect = 1
        return vect
    else:
      if x[-1] == vow[i]:
        vect = 1
        return vect
  return vect

In [ ]:
input_target['vect_vow'] = input_target.input.apply(vowel_features)

In [ ]:
# input_target['r_w'] = input_target.input.apply(r_or_w)

In [ ]:
input_target['vect'] = input_target.input.apply(features)

In [ ]:
input_target['plosives'] = input_target.input.apply(plosives)

In [ ]:
input_target.head()

,input,output,word_inp,word_tgt,label,prev,vect_vow,vect,plosives
0,_mɛ_,_mɛ̀_,mɛ,mɛ̀,1,"[0, 0, 0, 0, 0]","[0, 1]","[1, 1, 0]","[0, 0, 0, 0, 0, 0]"
1,_ɓot_,_ɓɔ̀t_,ɓot,ɓɔ̀t,1,"[0, 0, 0, 0, 0]","[0, 0]","[1, 1, 0]","[1, 0, 0, 0, 0, 0]"
2,_ɓô,_ɓo,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[0, 0]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]"
4,a_,a_,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[1, 1]","[0, 1, 1]","[0, 0, 0, 0, 0, 0]"
5,_n,_n,ni,ni,0,"[0, 0, 0, 0, 0]","[1, 1]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]"


# Transform vector into features

In [ ]:
def f_gen(x, ind):
  return x[ind]

In [ ]:
num = 1
col = ['prev', 'vect', 'vect_vow', 'r_w', 'next', 'plosives']

for c in col:
  if c == 'next' or c == 'r_w':
    continue
  tmp_l = len(input_target[c].iloc[0])
  print(tmp_l)
  for i in range(tmp_l):
    input_target['f'+str(num)] = input_target[c].apply(f_gen, ind=i)
    num += 1

5
3
2
6


In [ ]:
input_target.head()

,input,output,word_inp,word_tgt,label,prev,vect_vow,vect,plosives,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16
0,_mɛ_,_mɛ̀_,mɛ,mɛ̀,1,"[0, 0, 0, 0, 0]","[0, 1]","[1, 1, 0]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0
1,_ɓot_,_ɓɔ̀t_,ɓot,ɓɔ̀t,1,"[0, 0, 0, 0, 0]","[0, 0]","[1, 1, 0]","[1, 0, 0, 0, 0, 0]",0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0
2,_ɓô,_ɓo,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[0, 0]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,a_,a_,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[1, 1]","[0, 1, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0
5,_n,_n,ni,ni,0,"[0, 0, 0, 0, 0]","[1, 1]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0


In [ ]:
input_target['f'+str(num)] = input_target.input.apply(close)
# num += 1
# input_target['f'+str(num)] = input_target.input.apply(m)

In [ ]:
input_target['binary_label'] = input_target.label > 0

In [ ]:
input_target.head()

,input,output,word_inp,word_tgt,label,prev,vect_vow,vect,plosives,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,binary_label
0,_mɛ_,_mɛ̀_,mɛ,mɛ̀,1,"[0, 0, 0, 0, 0]","[0, 1]","[1, 1, 0]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,True
1,_ɓot_,_ɓɔ̀t_,ɓot,ɓɔ̀t,1,"[0, 0, 0, 0, 0]","[0, 0]","[1, 1, 0]","[1, 0, 0, 0, 0, 0]",0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,True
2,_ɓô,_ɓo,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[0, 0]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,False
4,a_,a_,ɓôdaa,ɓodàa,0,"[0, 0, 0, 0, 0]","[1, 1]","[0, 1, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,False
5,_n,_n,ni,ni,0,"[0, 0, 0, 0, 0]","[1, 1]","[1, 0, 1]","[0, 0, 0, 0, 0, 0]",0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,1,False


# Train the ML models

In [ ]:
base_X = input_target.drop(['input', 'output', 'word_inp', 'word_tgt', 'vect_vow', 'vect','label', 'prev',  'plosives', 'binary_label'], axis=1) #'r_w', 'next',
base_y = input_target.binary_label

In [ ]:
diac = input_target[input_target['binary_label']]

In [ ]:
X = diac.drop(['input', 'output', 'word_inp', 'word_tgt', 'vect_vow','vect','label', 'prev',  'plosives', 'binary_label'], axis=1) # 'next',, 'r_w'
y = diac.label

In [ ]:
df = input_target.drop(['input', 'output', 'word_inp', 'word_tgt', 'vect_vow', 'vect','prev', 'plosives'], axis=1) # 'next',, 'r_w'

In [ ]:
df.to_csv('features.csv', index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import numpy as np 

Add a simple logistic regression to predict whether we have to insert diacritic or not

In [ ]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(base_X, base_y, test_size = 0.2, random_state=21)

In [ ]:
logreg = LogisticRegression(max_iter=500, n_jobs=-1)

parameters = {'C': np.logspace(-5, 5, 20)}       

search = GridSearchCV(logreg, cv=5, param_grid = parameters, scoring = 'f1_macro')

# Fit it to the data
search.fit(X_train_b, y_train_b)
y_pred = search.predict(X_test_b)

# Compute and print metrics
print(classification_report(y_test_b, y_pred))
print("Tuned Model Parameters: {}".format(search.best_params_))

              precision    recall  f1-score   support

       False       0.61      0.73      0.66      4262
        True       0.61      0.48      0.54      3825

    accuracy                           0.61      8087
   macro avg       0.61      0.60      0.60      8087
weighted avg       0.61      0.61      0.60      8087

Tuned Model Parameters: {'C': 0.5455594781168515}


In [ ]:
logreg.set_params(**search.best_params_)
logreg.fit(base_X, base_y)

LogisticRegression(C=0.5455594781168515, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Here are more advanced functions to determine which diacritic we will need to add

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=21)

In [ ]:
logreg_features = LogisticRegression(C=0.01, max_iter=1000, n_jobs=-1, tol=0.001,
                            class_weight='balanced', 
                            solver='saga', multi_class = 'multinomial')
logreg_features.fit(X_train, y_train)
y_pred_q = logreg_features.predict(X_test)
print(classification_report(y_test, y_pred_q))

              precision    recall  f1-score   support

           1       0.86      0.02      0.04      2680
           2       0.02      0.95      0.03        22
           3       0.20      0.48      0.28       432
           4       0.27      0.38      0.31       594
           5       0.10      0.43      0.17       113

    accuracy                           0.15      3841
   macro avg       0.29      0.46      0.17      3841
weighted avg       0.66      0.15      0.11      3841



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
logreg_features.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='multinomial', n_jobs=-1,
                   penalty='l2', random_state=None, solver='saga', tol=0.001,
                   verbose=0, warm_start=False)

In [ ]:
%%time

sgd = SGDClassifier(alpha = 0.0001, max_iter=500, n_jobs=-1,  tol=0.0000001,
                    random_state=42, class_weight='balanced', loss= 'log')

# Fit it to the data
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

# Compute and print metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.77      0.79      0.78      2680
           2       0.00      0.00      0.00        22
           3       0.34      0.30      0.31       432
           4       0.42      0.19      0.26       594
           5       0.11      0.43      0.18       113

    accuracy                           0.63      3841
   macro avg       0.33      0.34      0.31      3841
weighted avg       0.64      0.63      0.62      3841

CPU times: user 1.02 s, sys: 16 ms, total: 1.04 s
Wall time: 634 ms


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.calibration import CalibratedClassifierCV

sgd.fit(X,y)
calibrator = CalibratedClassifierCV(sgd, cv='prefit')
calibrator.fit(X,y)

CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=0.0001, average=False,
                                                    class_weight='balanced',
                                                    early_stopping=False,
                                                    epsilon=0.1, eta0=0.0,
                                                    fit_intercept=True,
                                                    l1_ratio=0.15,
                                                    learning_rate='optimal',
                                                    loss='log', max_iter=500,
                                                    n_iter_no_change=5,
                                                    n_jobs=-1, penalty='l2',
                                                    power_t=0.5,
                                                    random_state=42,
                                                    shuffle=True, tol=1e-07,
                                       

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda = QuadraticDiscriminantAnalysis(reg_param=0.03)

qda.fit(X_train, y_train)
y_pred = qda.predict(X_test)
print(classification_report(y_test, y_pred))

qda.fit(X, y)

              precision    recall  f1-score   support

           1       0.72      0.78      0.75      2680
           2       0.06      0.95      0.12        22
           3       0.33      0.25      0.28       432
           4       0.49      0.09      0.15       594
           5       0.18      0.34      0.23       113

    accuracy                           0.60      3841
   macro avg       0.36      0.48      0.31      3841
weighted avg       0.62      0.60      0.58      3841



/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.03,
                              store_covariance=False, tol=0.0001)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()

lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.73      0.93      0.82      2680
           2       0.00      0.00      0.00        22
           3       0.39      0.19      0.26       432
           4       0.41      0.14      0.20       594
           5       0.00      0.00      0.00       113

    accuracy                           0.69      3841
   macro avg       0.31      0.25      0.26      3841
weighted avg       0.62      0.69      0.63      3841



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from collections import Counter

def predict(data):
  res = 0
  filter = logreg.predict(data)[0]
  
  if filter:
    out1 = logreg_features.predict(data)[0]
    out2 = calibrator.predict(data)[0]
    out3 = qda.predict(data)[0]
    out4 = lda.predict(data)[0]
    
    tot = [out1, out2, out3, out4]
    cnt = Counter(tot)
    res = [x[0] for x in cnt.most_common()][0]
  return res

In [ ]:
def make_df(sp, word_token, prev_d):

  feature_df = [0, 0, 0, 0, 0] #5
  feature_df.extend(vowel_features(sp)) #2
  # feature_df.extend(r_or_w(sp)) #3
  feature_df.extend(features(sp)) #5
  # feature_df.extend(next_diacritic(word_token, sp)) #5
  feature_df.extend(plosives(sp)) #6
  feature_df.append(close(sp)) #1
  # feature_df.append(m(sp)) #1

  if prev_d != 0:
    feature_df[prev_d-1] = 1
  X_dummies = pd.DataFrame([feature_df])
  
  return X_dummies

# Make predictions

In [ ]:
from tqdm.auto import tqdm 

In [ ]:
%%time

res = []

for i in tqdm(range(len(valid))):
  tmp = '' # our future line
  vowels = ['ı', 'ŋ', 'ɔ', 'ɛ', 'ñ', 'n', 'a', 'i', 'o', 'e', 'u', 'y']
  diacritics = ['̀', '́', '̂', '̄', '̌']
  funny_a = 'a' + '̀'

  line = valid.Protestant.iloc[i].split() # get the words out of the line
  for word in line:
    prev_diac = 0
    # check if the word is in our dictionary
    if word in no_change:
      tmp += no_change[word] + ' ' #add the part to the line
    else:      
      # apply pre-defined rules
      w = rules(word.lower())
      # make byte-pair encoding from the word and get rid of the punctuation
      word_token, splits = smart_split(str(word.lower()).strip(selected_punc))
      for sp in splits:     
        if sp in no_change_syll:
          v = no_change_syll[sp]
          sp = sp.replace('_', '')
          v = v.replace('_', '')
          w = w.replace(sp, v)

        else:          
          # create the vector
          tmp_df = make_df(sp, word_token, prev_diac)
          # make prediction
          pred =  predict(tmp_df)
          prev_diac = pred
          # replace _ with spaces to replace the parts in the actual word
          sp = sp.replace('_', '')
          # decode the prediction (replace the class number with corresponding diacritic
          vowel = ''
          # select the sound we want to apply the diacritic
          for v in vowels:
            if v in sp:
              vowel = v
          if vowel == 'i' and pred > 0:
            vowel = 'ı'
          if pred == 0 and vowel == 'o':
            w = w.replace('o', 'ɔ') 
          if pred:
            # pred -1 - is the index of the diacritic in the diacritics list
            rep = v + diacritics[pred-1]
            tmp_w = sp.replace(v, rep) #changed part
            w = w.replace(sp, tmp_w) #replace the part of the word with the part with diacritic
           
      tmp += w + ' ' 
  out = tmp.rstrip(' ')
  # out = out.replace('  ', ' ')
  res.append(out)


CPU times: user 22.4 s, sys: 251 ms, total: 22.7 s
Wall time: 22.9 s


In [ ]:
def one_to_two(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  new_w = x.replace('é', double_e).replace('ô', double_o).replace('ö', double_o_dots)
  return new_w

In [ ]:
import re

def diac_a_start(x):
  funny_a = ' '+'a'+ '̀' + ' '
  if x.startswith('A '):
    x = x.replace('A', 'À')
  if ' a ' in x:
    x = x.replace(' a ', funny_a)
  if '; mɛ' in x:
    x = x.replace('; mɛ', '; mɛ̀')
  if x.startswith('Mɛ '):
    x = x.replace('Mɛ ', 'Mɛ̀')     
  if re.search('(Ny|(by))ɛɛ', x):
    x = re.sub('((Ny)|B)ɛɛ', r'\1ɛ̀ɛ̀', x)
  if 'bɛɛ ' in x and not re.search('( |m)[̀́̂̄̌]?(b|ɓ)ɛɛ ', x):
    x = x.replace('bɛɛ ', 'ɓɛ̀ɛ̀ ') 
  if 'ôô' in x:
    x = x.replace('ôô', 'òò')
  if 'ɛ mɛ ' in x:
    x =  x.replace('ɛ mɛ', 'ɛ mɛ̀')
  if re.search('(K|( k)|( m))al ', x):
    x = re.sub('(K|( k)|( m))al ', r'\1ǎl ', x) # 'ŋɛ̀ŋ' 'ŋāŋ'
  if 'ŋaŋ' in x:
    x = x.replace('ŋaŋ', 'ŋāŋ')
  if 'ŋɛ̀ŋ' in x:
    x = x.replace('ŋɛŋ', 'ŋɛ̀ŋ')
  if ' lɛn' in x:
    x = x.replace(' lɛn',' lɛ̀n')
  if ' mut' in x:
    x = x.replace(' mut', ' mût')
  if x.startswith('Ŋk'):
    x =  x.replace('Ŋk', 'Ŋ̀k')
  if 'kum' in x:
    x = x.replace('kum', 'kùm')
  if 'koŋ' in x:
    x = x.replace('koŋ', 'kɔ̀ŋ')
  if 'Lim' in x:
    x = x.replace('Lim', 'Lı̀m')
  if 'kôt' in x:
    x = x.replace('kôt', 'kòt')
  if 'atɔh' in x:
    x = x.replace('atɔh', 'atɔ̄h') 
  if 'i lɔm' in x:
    x = x.replace('i lɔm','i lɔ̀m')
  if 'odɓ' in x:
    x = x.replace('odɓ','ɔ̄dɓ')
  return x

In [ ]:
out_dev = pd.DataFrame({'input': valid.Protestant, 'pred': res})
out_dev.pred = out_dev.pred.str.capitalize()
out_dev.pred = out_dev.pred.apply(diac_a_start)
valid.Academy = valid.Academy.apply(one_to_two)

out_dev.to_csv('out.tsv',sep='\t', index=False, header=False)
valid.to_csv('true_dev.tsv',sep='\t', index=False, header=False)

In [ ]:
valid.Academy = valid.Academy.apply(one_to_two)

out_dev.to_csv('out.tsv',sep='\t', index=False, header=False)
valid.to_csv('true_dev.tsv',sep='\t', index=False, header=False)

In [ ]:
out_dev.head()

,input,pred
0,Malét a mbôñe hyémbi.,Màlêt à mɓōŋɛ hyémbi.
1,A ntuhul me két (i) nuga.,À ntuhul mɛ kēt (i) nuga.
2,"Ñkum u ñkôs ndoñ, ñkum u nlôôs ndoñ.","Ŋ̀kùm u ŋkôs ndɔŋ, ŋ̀kùm u nlòòs ndɔŋ."
3,Mañ mut unu u nlo u mboda nyono wotama.,Maŋ mût unu u nlɔ u mɓoda nyɔnɔ wotama.
4,Boñge ba ñke jogop ba kahal ndugi kôt mañgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ jɔ̀gɔ̂p ɓa kahal ndugi kòt maŋgudgà.


In [ ]:
!python3 evaluate.py true_dev.tsv out.tsv

CER: 18.2899700164426
WER: 50.82327459294053


CER: 18.2899700164426

WER: 50.851449196115134

# Test data

In [ ]:
%%time

res_test = []

for i in tqdm(range(len(test))):
  tmp = '' # our future line
  vowels = ['ı', 'ŋ', 'ɔ', 'ɛ', 'ñ', 'n', 'a', 'i', 'o', 'e', 'u', 'y']
  diacritics = ['̀', '́', '̂', '̄', '̌']
  line = test.Protestant.iloc[i].split() # get the words out of the line
  for word in line:
    prev_diac = 0
    # check if the word is in our dictionary
    if word in no_change:
      tmp += no_change[word] + ' ' #add the part to the line

    else:      
      # apply pre-defined rules
      w = rules(word.lower())
      # make byte-pair encoding from the word and get rid of the punctuation
      word_token, splits = smart_split(str(word.lower()).strip(selected_punc))
      for sp in splits:     
        if sp in no_change_syll:
          v = no_change_syll[sp]
          sp = sp.replace('_', '')
          v = v.replace('_', '')
          w = w.replace(sp, v)

        else:          
          # create the vector
          tmp_df = make_df(sp, word_token, prev_diac)
          # make prediction
          pred =  predict(tmp_df)
          prev_diac = pred
          # replace _ with spaces to replace the parts in the actual word
          sp = sp.replace('_', '')
          # decode the prediction (replace the class number with corresponding diacritic
          vowel = ''
          # select the sound we want to apply the diacritic
          for v in vowels:
            if v in sp:
              vowel = v
          if vowel == 'i' and pred > 0:
            vowel = 'ı'
          if pred == 0 and vowel == 'o':
              w = w.replace('o', 'ɔ') 
          if pred:
            # pred -1 - is the index of the diacritic in the diacritics list
            rep = v + diacritics[pred-1]
            tmp_w = sp.replace(v, rep) #changed part
            w = w.replace(sp, tmp_w) #replace the part of the word with the part with diacritic
           
      tmp += w + ' ' 
  out = tmp.strip(' ')
  out = out.replace('  ', ' ')
  res_test.append(out)


CPU times: user 22.6 s, sys: 268 ms, total: 22.9 s
Wall time: 23 s


In [ ]:
out_test = pd.DataFrame({'input': test.Protestant, 'pred': res_test})
out_test.pred = out_test.pred.str.capitalize()
out_test.pred = out_test.pred.apply(diac_a_start)
test.Academy = test.Academy.apply(one_to_two)

In [ ]:
out_test.head()

,input,pred
0,Ndudu yem i kôli béé ni mut mpe.,Ndudu yɛ̂m i kôli ɓéé ni mût m̂pɛ.
1,Malép ma mpél podpot.,Màlep ma mpèl pɔdpɔt.
2,Béba mut i ñkéñha bape.,Ɓeba mût i ŋkéŋha ɓàpɛ.
3,Bitep bi yé we i jis.,Ɓitɛp bi yé wɛ i jis.
4,Hinjela hyem hi wom hi mput.,Hinjɛla hyɛ̂m hi wɔm hi mput.


In [ ]:
test.to_csv('true.tsv',sep='\t', index=False, header=False)
out_test.to_csv('out_test.tsv',sep='\t', index=False, header=False)

In [ ]:
!python3 evaluate.py true.tsv out_test.tsv

CER: 17.77620624256646
WER: 50.88492484124065


CER: 17.77620624256646

WER: 50.88492484124065

In [ ]:
!python3 /content/baseline/convert.py /content/baseline/model.tsv < /content/test.tsv > output_test.tsv

In [ ]:
!python3 evaluate.py test.tsv output_test.tsv

CER: 15.57748561509531
WER: 41.06452100531048


# Check

In [ ]:
baseline = pd.read_csv('/content/baseline/output.tsv', sep='\t', names = ['input', 'baseline'])

In [ ]:
baseline['true'] = valid.Academy.apply(one_to_two)
baseline['our_model'] = out_dev.pred

In [ ]:
b = baseline.drop('input', axis=1)

In [ ]:
b.iloc[[17, 197]]

,baseline,true,our_model
17,Nimblene mɛ̀ ŋkòò.,Nı̀mblɛnɛ mɛ̀ ŋ̀kòò.,Nimblɛnɛ mɛ̀ ŋkòò.
197,"À ntidha mɛ̀ ntuba ŋgı̀ minsôa, dı̀ ntidha makɔ̀ndɔ̀ mes.","À ǹtı̂dha mɛ ǹtùbà ŋgı̀ mı̀nsòa, dı̀ ǹtı̂dha makɔ̀ndɔ̀ mes.","À ntidha mɛ ntuba ŋgi minsôa, di ntidha makɔndɔ mes."


In [ ]:
baseline[baseline.input.str.contains('i lom')] 

,input,baseline,true,our_model
10,Kel i biyél me yani i lom.,Kɛl i biyél mɛ̀ yani i lɔ̂m.,Kɛl ı̀ biyēl mɛ yàni i lɔ̀m.,Kɛl i ɓiyél mɛ yani i lom.
703,Balop tjobi ba nha nsôñ i lom.,Balop cɔbı̄ ɓa ǹha ǹsôŋ i lɔ̂m.,Ɓàlɔp cɔ̀bi ɓa nha nsòŋ i lɔ̀m.,Ɓalɔp cobi ɓa nha nsôŋ i lom.
757,Nyemb i lép i mbés bôdaa ok i lom.,Nyɛ̌mb i lep i mbés ɓòdàa ɔ̄k i lɔ̂m.,Nyɛ̌mb ı̀ lep ı̀ mɓēs ɓodàà ɔk i lɔ̀m.,Nyɛ̌mb i lép i mɓés ɓôdàa ɔk i lom.


In [ ]:
baseline[baseline.true.str.contains('cɔ')] 

,input,baseline,true,our_model
109,E i nsayi= tjay tjéé di ñkudi tjodisô??,Ɛ i nsayi= cǎy cee dı̀ ŋkudı̄ tjodisô??,Ɛ i nsayı̂= cǎy cee di ŋkudı̄ cɔdisô??,Ɛ i nsayi= cay cee di ŋkudı̄ codisô??
155,Gwagwa le u somb tjobi ha nyaga béé.,Gwàgwà lɛ u sɔ̄mb cɔbı̄ ha nyàgà ɓee.,Gwàgwà lɛ ù sɔmb cɔbı̄ hà nyàgà ɓee.,Gwàgwà lɛ u sɔmb cobi ha nyaga ɓéé.
158,Hitjo hi mbodbe nye ni dinoo .,Hı̀cɔ hi mbodbe nyɛ nı̀ dinɔ̀ɔ .,Hı̀cɔ hi mɓɔ̄dɓɛ nyɛ nı̀ dı̀nɔ̀ɔ .,Hicɔ hi mɓɔ̄dɓɛ nyɛ ni dinɔ̀ɔ .
224,Nuu mañge a bitjogop ñgandak.,Nuu màŋgɛ à bitjogop ŋgandàk.,Nuu màŋgɛ à ɓicɔ̄gɔp ŋgandàk.,Nuu maŋgɛ à ɓicogɔ̂p ŋgandàk.
280,Nyañgô a nnéyés dibee tjodisô .,Nyàŋgo à nnéyés diɓɛ̀ɛ cɔdisô .,Nyàŋgo à ǹneyēs diɓɛ̀ɛ̀ cɔdisô .,Nyàŋgo à nnéyés diɓɛ̀ɛ̀ cɔdisô .
350,Ke tjo kabi.,Kɛ̀ cɔ kabi.,Kɛ̀ cɔ kàbi.,Kɛ̀ cɔ kabi.
458,Ba nligis hitjo i ndap Jop.,Ɓa nligis hı̀cɔ i ndap Jɔ̌p.,Ɓa nlı̄gis hicɔ i ndāp Jɔ̌p.,Ɓa nligis hicɔ i ndap jɔ̌p.
703,Balop tjobi ba nha nsôñ i lom.,Balop cɔbı̄ ɓa ǹha ǹsôŋ i lɔ̂m.,Ɓàlɔp cɔ̀bi ɓa nha nsòŋ i lɔ̀m.,Ɓalɔp cobi ɓa nha nsôŋ i lom.
872,Njéé i njena; basaa ba njena kii tjobi.,Njèe i njena; ɓasàa ɓa njɛ̄na kı̀i cɔ̀bi.,Njèe ı̀ njɛ̄na; ɓàsàa ɓa njɛ̄na kı̀i cɔ̀bi.,Njèe i njɛna; ɓasaa ɓa njɛ̄na kı̀i cobi.


In [ ]:
baseline[baseline.true.str.contains('m̂ɓendha')] 

,input,baseline,true,our_model
993,U mbéndha me mapep.,Ù m̂ɓêndhā mɛ̀ mapɛp.,Ù m̂ɓendha mɛ màpɛp.,U m̂ɓêndhā mɛ màpɛp.


In [ ]:
b[b.our_model.str.contains('co')]

,baseline,true,our_model
109,Ɛ i nsayi= cǎy cee dı̀ ŋkudı̄ tjodisô??,Ɛ i nsayı̂= cǎy cee di ŋkudı̄ cɔdisô??,Ɛ i nsayi= cay cee di ŋkudı̄ codisô??
155,Gwàgwà lɛ u sɔ̄mb cɔbı̄ ha nyàgà ɓee.,Gwàgwà lɛ ù sɔmb cɔbı̄ hà nyàgà ɓee.,Gwàgwà lɛ u sɔmb cobi ha nyaga ɓéé.
224,Nuu màŋgɛ à bitjogop ŋgandàk.,Nuu màŋgɛ à ɓicɔ̄gɔp ŋgandàk.,Nuu maŋgɛ à ɓicogɔ̂p ŋgandàk.
703,Balop cɔbı̄ ɓa ǹha ǹsôŋ i lɔ̂m.,Ɓàlɔp cɔ̀bi ɓa nha nsòŋ i lɔ̀m.,Ɓalɔp cobi ɓa nha nsôŋ i lom.
872,Njèe i njena; ɓasàa ɓa njɛ̄na kı̀i cɔ̀bi.,Njèe ı̀ njɛ̄na; ɓàsàa ɓa njɛ̄na kı̀i cɔ̀bi.,Njèe i njɛna; ɓasaa ɓa njɛ̄na kı̀i cobi.


In [ ]:
b[b.true.str.contains('ɔ̄dɓ')]

,baseline,true,our_model
158,Hı̀cɔ hi mbodbe nyɛ nı̀ dinɔ̀ɔ .,Hı̀cɔ hi mɓɔ̄dɓɛ nyɛ nı̀ dı̀nɔ̀ɔ .,Hicɔ hi mɓodɓɛ nyɛ ni dinɔ̀ɔ .


In [ ]:
def word_error(true, x):
  l_t = true.split()
  l_x = x.split()
  er = 0
  if len(l_x) < len(l_t):
    er += len(l_t) - len(l_x)
    for i in range(len(l_x)):
      if l_t[i] != l_x[i]:
        er += 1
  else:
     for i in range(len(l_t)):
      if l_t[i] != l_x[i]:
        er += 1
  return er

In [ ]:
c = baseline.drop('baseline', axis=1)

c['our_model_word_error'] = c.apply(lambda x: word_error(x['true'], x['our_model']), axis=1)
c.to_excel('report.xlsx')

In [ ]:
b['our_model_word_error'] = b.apply(lambda x: word_error(x['true'], x['our_model']), axis=1)
b['base_model_word_error'] = b.apply(lambda x: word_error(x['true'], x['baseline']), axis=1)

In [ ]:
b[b['our_model_word_error'] <= b['base_model_word_error']] 

,baseline,true,our_model,our_model_word_error,base_model_word_error
1,À ntuhul mɛ̀ kēt (i) nugā.,À ǹtuhul mɛ kēt (i) nugā.,À ntuhul mɛ kēt (i) nuga.,2,2
2,"Ñkum u ŋ̀kôs ndoñ, ŋ̀kùm u ǹloōs ndɔŋ.","Ŋ̀kùm u ŋkòs ndɔŋ, ŋ̀kùm u nlòos ndɔŋ.","Ŋ̀kùm u ŋkôs ndɔŋ, ŋ̀kùm u nlòòs ndɔŋ.",2,4
3,Maŋ mùt unu u nlɔ̀ u mboda nyɔ̀nɔ wotama.,Maŋ mût unu u nlɔ ū mɓɔ̄da nyɔnɔ̄ wɔtāma.,Maŋ mût unu u nlɔ u mɓoda nyɔnɔ wotama.,4,6
4,Ɓɔ̀ŋgɛ ɓa ŋkɛ̀ jɔ̀gɔ̂p ɓa kahal ndugi kòt mañgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ̂ jɔ̀gɔ̂p ɓa kahal ndugi kòt maŋgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ jɔ̀gɔ̂p ɓa kahal ndugi kòt maŋgudgà.,2,2
5,Bihégél bi ndap u ǹlama yɔŋ yı̄hɛ lɛ jàm li sɛt ɓâŋ.,Bihègel bi ndap ù nlama yɔ̂ŋ yihɛ lɛ jàm li sɛ̀t ɓâŋ.,Ɓihégél bi ndap u nlama yoŋ yihɛ lɛ jàm li sɛt ɓâŋ.,4,6
...,...,...,...,...,...
993,Ù m̂ɓêndhā mɛ̀ mapɛp.,Ù m̂ɓendha mɛ màpɛp.,U m̂ɓêndhā mɛ mapɛp.,3,3
996,Maŋ mùt unu u ñeñ màen hi kôkôwa.,Maŋ mût unu u ŋɛ̀ŋ maen hi kokōwa.,Maŋ mût unu u ŋɛŋ màen hi kôkôwà.,3,4
997,Limbee li mût.,Lı̀mbɛ̀ɛ̀ li mût.,Lı̀mbɛɛ li mût.,1,1
998,"Ndi wɛ̂, ŋgèdà u nsôga jɛ, hɔ̄ɔ ŋɔ wɔŋ mòo, u sɔ̀ɔ ki su woñ...(Mt 6,17).","Ndi wɛ̀, ŋgèdà ù nsōga jɛ, hɔɔ ŋ̀ɔ wɔŋ mòo, u sɔɔ̄ kı̀ su wɔŋ...(Mt 6,17).","Ndi wɛ, ŋgèdà u nsôga jɛ, hɔɔ ŋɔ woŋ mòo, u sɔ̀ɔ ki su woŋ...(mt 6,17).",8,8


In [ ]:
b.to_excel('report.xlsx')

In [ ]:
b.head()

,baseline,true,our_model,our_model_word_error,base_model_word_error
0,Màlêt à mɓōŋɛ hyembi.,Màlêt à m̂ɓoŋɛ hyembi.,Màlêt a mɓōŋɛ hyémbi.,3,1
1,À ntuhul mɛ̀ kēt (i) nugā.,À ǹtuhul mɛ kēt (i) nugā.,À ntuhul mɛ kēt (i) nuga.,2,2
2,"Ñkum u ŋ̀kôs ndoñ, ŋ̀kùm u ǹloōs ndɔŋ.","Ŋ̀kùm u ŋkòs ndɔŋ, ŋ̀kùm u nlòos ndɔŋ.","Ŋkum u ŋkôs ndɔŋ, ŋ̀kùm u nlôôs ndɔŋ.",3,4
3,Maŋ mùt unu u nlɔ̀ u mboda nyɔ̀nɔ wotama.,Maŋ mût unu u nlɔ ū mɓɔ̄da nyɔnɔ̄ wɔtāma.,Maŋ mut unu u nlɔ u mɓoda nyɔnɔ wotama.,5,6
4,Ɓɔ̀ŋgɛ ɓa ŋkɛ̀ jɔ̀gɔ̂p ɓa kahal ndugi kòt mañgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ̂ jɔ̀gɔ̂p ɓa kahal ndugi kòt maŋgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ jɔ̀gɔ̂p ɓa kahal ndugi kôt maŋgudgà.,3,2


# Ensemble

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mbart_5.tsv to mbart_5.tsv
Saving mbart_5_test.tsv to mbart_5_test.tsv


## Dev

In [ ]:
out1 = pd.read_csv('/content/baseline/output.tsv', sep ='\t', names = ['Protestant', 'Academy'])
out2 = pd.read_csv('out.tsv', sep ='\t', names = ['Protestant', 'Academy'])
out3 = pd.read_csv('mbart_5.tsv', sep ='\t', names = ['Protestant', 'Academy'])

In [ ]:
def one_to_two(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  new_w = x.replace('é', double_e).replace('ô', double_o).replace('ö', double_o_dots)
  return new_w

In [ ]:
out1.Academy = out1.Academy.apply(one_to_two)
out2.Academy = out2.Academy.apply(one_to_two)
out3.Academy = out3.Academy.apply(one_to_two)

In [ ]:
def spl(x):
  return x.split()

In [ ]:
out1.Academy = out1.Academy.apply(spl)
out2.Academy = out2.Academy.apply(spl)
out3.Academy = out3.Academy.apply(spl)

In [ ]:
def zoo(out1, *args):
  res = []
  x = [arg for arg in args if len(arg)>=len(out1)]
  for w in range(len(out1)):
    ww = []
    ww.append(out1[w])
    for arg in args:
      if arg in x:
        ww.append(arg[w])
    tmp = c(*ww)
    res.append(tmp)
  return ' '.join(res)

In [ ]:
from collections import Counter
def c(*args):
  words = []
  for arg in args:
    words.append(arg)
  cnt = Counter(words)
  return [x[0] for x in cnt.most_common()][0]

In [ ]:
a = []

for i in range(1000):
  tmp = zoo(out1.Academy.to_list()[i], out2.Academy.to_list()[i], out3.Academy.to_list()[i])
  a.append(tmp)

In [ ]:
df1 =  pd.DataFrame({'input': valid['Protestant'], 'pred':a})
df1.to_csv('out_ans.tsv',sep='\t', index=False, header=False)

In [ ]:
test = test.drop(test[test.Protestant == '?'].index)

In [ ]:
valid.Academy = valid.Academy.apply(one_to_two)
valid.to_csv('valid.tsv',sep='\t', index=False, header=False)

In [ ]:
!python3 evaluate.py valid.tsv out_ans.tsv

CER: 15.07237966276558
WER: 39.15383098251907


## Test

In [ ]:
out1_test = pd.read_csv('output_test.tsv', sep ='\t', names = ['Protestant', 'Academy'])
out2_test = pd.read_csv('out_test.tsv', sep ='\t', names = ['Protestant', 'Academy'])
out3_test = pd.read_csv('mbart_5_test.tsv', sep ='\t', names = ['Protestant', 'Academy'])

In [ ]:
out1_test.Academy = out1_test.Academy.apply(one_to_two)
out2_test.Academy = out2_test.Academy.apply(one_to_two)
out3_test.Academy = out3_test.Academy.apply(one_to_two)

In [ ]:
out1_test.Academy = out1_test.Academy.apply(spl)
out2_test.Academy = out2_test.Academy.apply(spl)
out3_test.Academy = out3_test.Academy.apply(spl)

In [ ]:
a = []

for i in range(1000):
  tmp = zoo(out1_test.Academy.to_list()[i], out2_test.Academy.to_list()[i], out3_test.Academy.to_list()[i])
  a.append(tmp)

In [ ]:
df1_test =  pd.DataFrame({'input': test['Protestant'], 'pred':a})
df1_test.to_csv('out_ans_test.tsv',sep='\t', index=False, header=False)

In [ ]:
test.Academy = test.Academy.apply(one_to_two)
test.to_csv('test.tsv',sep='\t', index=False, header=False)

In [ ]:
!python3 evaluate.py test.tsv out_ans_test.tsv

CER: 14.153460413385194
WER: 39.51744127948075
